In [2]:
import cv2
import torch
import torchvision.transforms as transforms
import onnx
import onnxruntime as ort

import models
from utils.eval import measure_time_onnx, measure_time_torch

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# model
model = models.Model(pretrained=False)
model.load_state_dict(torch.load('trained1_state.pth', map_location='cpu'))

# data
transform = transforms.ToTensor()
data = transform(cv2.resize(cv2.imread("./outputs/test.png"), (224, 224)))[None, ...]
data.shape

# iters
n_iter = 1000

# Pytorch personal PC

## Cuda

In [22]:
device = 'cuda:0'
res = measure_time_torch(n_iter, data, model, device)
res

22.09052610397339

## CPU

In [23]:
device = 'cpu'
res = measure_time_torch(n_iter, data, model, device)
res

158.67240381240845

# ONNX personal PC

## CPU

In [25]:
torch.onnx.export(
        model, data, "fast_deep.onnx",
        input_names=['image'], output_names=['mask']
)

model_onnx_spec = onnx.load('fast_deep.onnx')
onnx.checker.check_model(model_onnx_spec)

ort_sess = ort.InferenceSession(
    'fast_deep.onnx', 
    providers=['CPUExecutionProvider']
)

/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/.venv/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/.venv/lib/python3.10/site-packages/torch/onnx/utils.py:687: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/home/alexander/computerScience/phystech/9sem/abbyy/course_cvdl/.venv/li

In [40]:
res = measure_time_onnx(n_iter, data, ort_sess)
res

94.53361821174622

## Cuda

In [42]:
ort_inference_session=ort.InferenceSession(
        'fast_deep.onnx', 
        providers=['CUDAExecutionProvider']
    )

In [43]:
res = measure_time_onnx(n_iter, data, ort_inference_session)
res

20.936627626419067

# Torch RPi

In [ ]:
device = 'cpu'
res = measure_time_torch(n_iter, data, model, device)
res

this makes jupyter kernel die

# Onnx RPi

In [4]:
model_onnx_spec = onnx.load('fast_deep.onnx')
onnx.checker.check_model(model_onnx_spec)

ort_sess = ort.InferenceSession(
    'fast_deep.onnx', 
    providers=['CPUExecutionProvider']
)

In [5]:
res = measure_time_onnx(n_iter, data, ort_sess)
res

2516.317708969116